In [1]:
import csv
import urllib2    
from collections import defaultdict
import time
from urllib import FancyURLopener  # This is library that helps us create the headless browser
from random import choice #This library helps pick a random item from a list
import pandas as pd
import string

In [2]:
import bs4 as bs
from bs4 import SoupStrainer

In [3]:
## Creating a list to help in pagination
doc_list = range(0,80)
doc_list = [((i * 100)+1) for i in doc_list]

In [4]:
## Links to the website that gives the 13f links
link_2016 = []
for i in doc_list:
    link_2016.append('https://searchwww.sec.gov/EDGARFSClient/jsp/EDGAR_MainAccess.jsp?search_text=Information+table&sort=Date&startDoc='+str(i)+'&numResults=100&isAdv=true&formType=Form13FHR&fromDate=12/31/2013&toDate=12/01/2016&stemming=true')

In [5]:
link_2015 = []
for i in doc_list:
    link_2015.append('https://searchwww.sec.gov/EDGARFSClient/jsp/EDGAR_MainAccess.jsp?search_text=Information+table&sort=Date&startDoc='+str(i)+'&numResults=100&isAdv=true&formType=Form13FHR&fromDate=12/31/2013&toDate=12/01/2015&stemming=true')

In [6]:
link_2014 = []
for i in doc_list:
    link_2014.append('https://searchwww.sec.gov/EDGARFSClient/jsp/EDGAR_MainAccess.jsp?search_text=Information+table&sort=Date&startDoc='+str(i)+'&numResults=100&isAdv=true&formType=Form13FHR&fromDate=12/31/2013&toDate=12/01/2014&stemming=true')

In [7]:
urls = link_2016 + link_2015 + link_2014

In [8]:
user_agents = [
    'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36',
    'Opera/9.80 (X11; Linux i686; Ubuntu/14.10) Presto/2.12.388 Version/12.16',
    'Mozilla/5.0 (Windows; U; Windows NT 6.1; rv:2.2) Gecko/20110201',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.75.14 (KHTML, like Gecko) Version/7.0.3 Safari/7046A194A',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246'
]

In [9]:
## Lists to populate
CIK_list = []
date_list = []
link_list = []

## Looping through all the links and getting the 13f links
for j in urls:
    
    class MyOpener(FancyURLopener, object):
        version = choice(user_agents)


    myopener = MyOpener()
    page=myopener.open(j)    
    html = page.read()
    

    soup = bs.BeautifulSoup(html,"lxml")

    date = soup.find_all("i", class_="blue")

    link13f = soup.find_all("a",class_ = "filing")

    CIK = soup.find_all("a",id = "cikSearch")

    for i in range(0,len(CIK)):
        date_list.append(date[i].getText())
        CIK_list.append(CIK[i].getText())

        start = link13f[i]["href"].find("http")
        end = link13f[i]["href"].find(",")

        link_list.append(link13f[i]["href"][start:end-1])

In [10]:
link_list

['http://www.sec.gov/Archives/edgar/data/1672681/000121465916015098/xslForm13F_X01/infotable.xml',
 'http://www.sec.gov/Archives/edgar/data/1672681/000121465916015099/xslForm13F_X01/infotable.xml',
 'http://www.sec.gov/Archives/edgar/data/1672681/000121465916015100/xslForm13F_X01/infotable.xml',
 'http://www.sec.gov/Archives/edgar/data/1672681/000121465916015101/xslForm13F_X01/infotable.xml',
 'http://www.sec.gov/Archives/edgar/data/1672681/000121465916015102/xslForm13F_X01/infotable.xml',
 'http://www.sec.gov/Archives/edgar/data/1672681/000121465916015103/xslForm13F_X01/infotable.xml',
 'http://www.sec.gov/Archives/edgar/data/1672681/000121465916015104/xslForm13F_X01/infotable.xml',
 'http://www.sec.gov/Archives/edgar/data/1672681/000121465916015105/xslForm13F_X01/infotable.xml',
 'http://www.sec.gov/Archives/edgar/data/1672681/000121465916015106/xslForm13F_X01/infotable.xml',
 'http://www.sec.gov/Archives/edgar/data/1689872/000139834416021322/xslForm13F_X01/fp0022662_13fhr-table.xml'

In [11]:
## Getting the dataset with links to write out
df_out = pd.DataFrame({'Date': date_list,'CIK': CIK_list,'13Fweblink': link_list})

In [12]:
df_out.to_csv("13Flinks.csv")
## The file is opened in excel and subsetted only for those mutual fund CIKs which have data for three continuos years
## Then we used random sampling to select 971(30% of all available) mutual funds